<h1 style="text-align:center;font-size:30px;" > Quora Question Pair Similarity Continued </h1>

### Notebook - 3

## Text Featurization using Tf-IDF and Glove

In [1]:
# first things first loadinf some basic packages
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm 
# Libraries for text featurization
from sklearn.feature_extraction.text import TfidfVectorizer
import spacy
import en_core_web_lg
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Loading dataset
data = pd.read_csv("2_advance_feature_preprocessing_extraction.csv")
data = data.fillna(" ")

In [3]:
# getting cleaned questions into sepearte numpy array

question_1 = data.question1
question_2 = data.question2

## Text Featurization using Tf-IDF 

In [5]:
# getting all question into single list
all_questions = np.dstack([question_1, question_2]).flatten()

# Instantiating Tf-IDF Vectorizer
tfidf = TfidfVectorizer(ngram_range=(1,1),min_df=10)

tf_idf_bow = tfidf.fit_transform(all_questions)

# zipping together each word and its tf_idf value
tf_idf = dict(zip(tfidf.get_feature_names(), tfidf.idf_))

## Text Featurization using Tf-IDF_Glove

In [6]:
# loading spacy model
nlp = en_core_web_lg.load()

In [9]:

def glove_vect(question):
    
    sentence_vectors =[]
    """ LOOP for each question"""
    for sentence in tqdm(question):
        doc = nlp(sentence)
        sent_avg_vec = np.zeros(len(doc[0].vector))
        weight_sum = 0
        
        """ LOOP for each word in question"""
        for token in doc:
            if token.has_vector and str(token) in tf_idf:
                glove_vec = token.vector
                tfidf = sentence.count(str(token))/len(sentence)*tf_idf.get(str(token))
                vector = glove_vec*tfidf
                sent_avg_vec+= vector 
                weight_sum += tfidf
        
        if weight_sum != 0:
            sent_avg_vec /= weight_sum
            
        sentence_vectors.append(sent_avg_vec)
    
    return sentence_vectors
                
    

In [10]:
question_1_vector = glove_vect(question_1)

In [11]:
question_2_vector = glove_vect(question_2)

In [18]:
# Converting crated features from list to DataFrame
index = ["X_"+str(x) for x in range(1,301)]
q1_vector = pd.DataFrame(question_1_vector, columns = index)

index = ["Y_"+str(x) for x in range(1,301)]
q2_vector = pd.DataFrame(question_2_vector, columns = index)

In [19]:
# Merging two dataframes
question_vectors = pd.concat([q1_vector, q2_vector], axis=1)

In [20]:
question_vectors.shape

(404290, 600)

In [21]:
# Saving created features of text

question_vectors.to_csv("Glove_Vectors.csv", index=False)

In [297]:
import en_core_web_lg
# import en_vectors_web_lg
n = en_core_web_lg.load()

In [22]:
# Combining all the features into single dataframe
final_features = pd.concat([data, question_vectors], axis=1)

In [25]:
# Saving Final_features
final_features.to_csv("final_features.csv", index=False)